In [ ]:
import os
import json
import torch
import torchvision
import torch.nn as nn
import numpy as np
import cv2
import gradio as gr
from skimage.feature import graycomatrix, graycoprops
from sklearn.ensemble import RandomForestClassifier
import joblib

In [ ]:
json_path = r"D:\MED_LEAF_ID\class_labels.json"

In [ ]:
if not os.path.exists(json_path):
    print("⚠️ class_labels.json not found! Generating...")
    dataset_path = r"D:\MED_LEAF_ID\dataset\Medicinal Leaf dataset"
    class_labels = {str(idx): class_name for idx, class_name in enumerate(sorted(os.listdir(dataset_path)))}
    with open(json_path, "w") as f:
        json.dump(class_labels, f, indent=4)
    print(f"✅ Class labels saved to {json_path}")
else:
    with open(json_path, "r") as f:
        class_labels = json.load(f)

In [ ]:
def get_class_name(pred_class):
    return class_labels.get(str(pred_class), "Unknown Class")


In [ ]:
class EfficientNetB0Classifier(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetB0Classifier, self).__init__()
        self.base_model = torchvision.models.efficientnet_b0(pretrained=True)
        self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, num_classes)
        self.base_model.classifier.add_module('dropout', nn.Dropout(0.2))

    def forward(self, x):
        return self.base_model(x)


In [ ]:
cnn_model_path = r"D:\MED_LEAF_ID\models\efficientnetb0_leaf_model.pth"
cnn_model = EfficientNetB0Classifier(num_classes=80)
cnn_model.load_state_dict(torch.load(cnn_model_path, map_location="cpu"))
cnn_model.eval()

In [ ]:
rf_model_path = r"D:\MED_LEAF_ID\models\rf_glcm_model.pkl"
rf_model = joblib.load(rf_model_path)

In [ ]:
def extract_glcm_features(image):
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        glcm = graycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
        features = np.hstack([
            graycoprops(glcm, 'contrast').flatten(),
            graycoprops(glcm, 'dissimilarity').flatten(),
            graycoprops(glcm, 'homogeneity').flatten(),
            graycoprops(glcm, 'energy').flatten(),
            graycoprops(glcm, 'correlation').flatten(),
            graycoprops(glcm, 'ASM').flatten(),
        ])
        return features
    except Exception as e:
        return None

In [ ]:
def predict(image):
    try:
        # CNN Prediction
        img_resized = cv2.resize(image, (224, 224)) / 255.0
        img_tensor = torch.tensor(img_resized, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)
        cnn_pred = torch.argmax(cnn_model(img_tensor)).item()

        # GLCM + RandomForest Prediction
        glcm_features = extract_glcm_features(image)
        if glcm_features is None or len(glcm_features) != 30:
            return "Error: GLCM feature extraction failed!"

        rf_pred = rf_model.predict([glcm_features])[0]

        # Weighted Ensemble (70% CNN, 30% RF)
        final_pred = cnn_pred if np.random.rand() < 0.7 else rf_pred
        plant_name = get_class_name(final_pred)

        return f"Predicted Plant: {plant_name}"
    except Exception as e:
        return f"Error in prediction: {str(e)}"

In [ ]:
import json

# Load class labels
json_path = r"D:\MED_LEAF_ID\class_labels.json"
with open(json_path, "r") as f:
    class_mapping = json.load(f)

# Example: Your model predicts class index 0
predicted_class_index = 0  # Replace this with your actual model prediction

# Get the plant name
plant_name = class_mapping.get(str(predicted_class_index), "Unknown")

print(f"🌿 Predicted Plant Name: {plant_name}")


In [ ]:
iface = gr.Interface(fn=predict, inputs=gr.Image(type="numpy"), outputs="text")
iface.launch()